In [1]:
import pandas as pd
import json

In [6]:
cve_data = pd.DataFrame.from_csv('cve_data.csv')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [7]:
cve_data

,CVE_ID,CWE,accessVector,accessComplexity,authentication,confidentialityImpact,integrityImpact,availabilityImpact,baseScore,severity,exploitabilityScore,impactScore,publishedDate,lastModifiedDate,CWE_Description
0,CVE-1999-0001,CWE-20,NETWORK,LOW,NONE,NONE,NONE,PARTIAL,5.0,MEDIUM,10.0,2.9,1999-12-30T05:00Z,2010-12-16T05:00Z,Input Validation
1,CVE-1999-0002,CWE-119,NETWORK,LOW,NONE,COMPLETE,COMPLETE,COMPLETE,10.0,HIGH,10.0,10.0,1998-10-12T04:00Z,2009-01-26T05:00Z,Buffer Errors
2,CVE-1999-0003,NVD-CWE-Other,NETWORK,LOW,NONE,COMPLETE,COMPLETE,COMPLETE,10.0,HIGH,10.0,10.0,1998-04-01T05:00Z,2018-10-30T16:26Z,Other
3,CVE-1999-0004,NVD-CWE-Other,NETWORK,LOW,NONE,NONE,NONE,PARTIAL,5.0,MEDIUM,10.0,2.9,1997-12-16T05:00Z,2018-10-12T21:29Z,Other
4,CVE-1999-0005,NVD-CWE-Other,NETWORK,LOW,NONE,COMPLETE,COMPLETE,COMPLETE,10.0,HIGH,10.0,10.0,1998-07-20T04:00Z,2008-09-09T12:33Z,Other
5,CVE-1999-0006,NVD-CWE-Other,NETWORK,LOW,NONE,COMPLETE,COMPLETE,COMPLETE,10.0,HIGH,10.0,10.0,1998-07-14T04:00Z,2008-09-09T12:33Z,Other
6,CVE-1999-0007,NVD-CWE-Other,NETWORK,LOW,NONE,PARTIAL,NONE,NONE,5.0,MEDIUM,10.0,2.9,1998-06-26T04:00Z,2018-10-12T21:29Z,Other
7,CVE-1999-0008,NVD-CWE-Other,NETWORK,LOW,NONE,COMPLETE,COMPLETE,COMPLETE,10.0,HIGH,10.0,10.0,1998-06-08T04:00Z,2018-10-30T16:26Z,Other
8,CVE-1999-0009,NVD-CWE-Other,NETWORK,LOW,NONE,COMPLETE,COMPLETE,COMPLETE,10.0,HIGH,10.0,10.0,1998-04-08T04:00Z,2018-10-30T16:26Z,Other
9,CVE-1999-0010,NVD-CWE-Other,NETWORK,LOW,NONE,NONE,NONE,PARTIAL,5.0,MEDIUM,10.0,2.9,1998-04-08T04:00Z,2018-10-30T16:26Z,Other
